### Imports & Definitions

In [11]:
import sys
sys.path.append("../../utils")
from definitions import *
from path_helpers import get_dataset_path, get_metric_dir_path

### Process results

In [12]:
rows = []
for baseline_model in HUGGINFACE_BASELINE_MODELS.union([VIKHR_7B, SAIGA_MISTRAL_7B_LORA]):
    for name, dataset_meta in HUGGINGFACE_NAME_TO_DATASET.items():
        for subset, split in zip(dataset_meta["subsets"], dataset_meta["splits"]):
            dataset_path = get_dataset_path(subset, name, split)
            dataset = load_from_disk(dataset_path)
            metric_path = get_metric_dir_path(baseline_model, subset, name, split)
            with open(metric_path.joinpath("answers.jsonl"), "r") as f:
                answers = json.load(f)
            pred = [x["answer"] for x in answers]
            true = [x["outputs"] for x in dataset]
            rows.append({
                "model": baseline_model,
                "subset": subset,
                "split": split,
                "accuracy_score": accuracy_score(true, pred),
            })
df = pd.DataFrame(rows)
df.to_csv(METRICS_DIR_PATH/"ru_metrics.csv")
df

,model,subset,split,accuracy_score
0,saiga_mistral_7b_lora,rummlu,public_test,0.447424
1,saiga_mistral_7b_lora,ruopenbookqa,train,0.602652
2,vikhr-7b,rummlu,public_test,0.309279
3,vikhr-7b,ruopenbookqa,train,0.336185
4,mistral-7b,rummlu,public_test,0.423203
5,mistral-7b,ruopenbookqa,train,0.525663
6,llama-2-7b,rummlu,public_test,0.301106
7,llama-2-7b,ruopenbookqa,train,0.262618
